In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from transformers import Trainer
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, get_scheduler
from transformers import AdamW
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import resample
from tqdm import tqdm
import wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd_orientation = pd.read_csv('drive/MyDrive/NLP_THE2/orientation-tr-train.tsv', sep='\t')

In [ ]:
pd_orientation.head()

,id,speaker,sex,text,text_en,label
0,tr00000,ca2031caa4032c51980160359953d507,M,"Yeni yasama döneminin ülkemiz için, milletimiz...","Mr. President, dear lawmakers, I salute you, a...",1
1,tr00001,4cee0addb3c69f6866869b180f90d45f,M,"Sayın Başkan, değerli milletvekilleri; bugün, ...","Mr. President, members of lawmakers, as I spea...",1
2,tr00002,b3d7f76d74ec268492f8190ca123a6b2,M,"Sayın Başkanım, öncelikle yüce Meclisin Başkan...","Mr. President, I'm here to share with you the ...",1
3,tr00003,722efac7138c8197a9d1e97eed3a8b18,M,24’üncü Dönem Meclis Başkanlığına seçilmenizde...,"Mr. President, under the principles determined...",1
4,tr00004,be82a4ade406ec6774a0a2e38f6957e3,M,24’üncü Yasama Dönemimizin tüm milletvekilleri...,"Mr. President, dear lawmakers, I ask God's mer...",1


In [ ]:
majority = pd_orientation[pd_orientation['label'] == 1]
minority = pd_orientation[pd_orientation['label'] == 0]
minority_oversampled = resample(
    minority,
    replace=True,
    n_samples=len(majority),
    random_state=42
)
oversampled_data = pd.concat([majority, minority_oversampled])
pd_orientation = oversampled_data

In [ ]:
X = pd_orientation['text']  # Using Turkish translations
y = pd_orientation['label']    # Labels (0 for left, 1 for right)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.10,
    random_state=42,
    stratify=y    # This ensures proportional split of labels
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Training set size: 16902
Test set size: 1878


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [ ]:
max_length = 512

def tokenize_data(texts, tokenizer, max_length):
    return tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )


train_encodings = tokenize_data(X_train, tokenizer, max_length)
test_encodings = tokenize_data(X_test, tokenizer, max_length)


In [ ]:
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)


class TurkishParliamentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: tensor[idx] for key, tensor in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# Create datasets
train_dataset = TurkishParliamentDataset(train_encodings, train_labels)
test_dataset = TurkishParliamentDataset(test_encodings, test_labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=2  # Binary classification: 0 (left), 1 (right)
)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average="binary")
    recall = recall_score(labels, preds, average="binary")
    f1 = f1_score(labels, preds, average="binary")
    accuracy = accuracy_score(labels, preds)

    # Log metrics to WandB
    wandb.log({
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy
    })

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at regular intervals
    eval_steps=500,               # Frequency of evaluation
    learning_rate=2e-5,           # Initial learning rate
    lr_scheduler_type="linear", # Gradually reduce learning rate
    per_device_train_batch_size=32,  # Increase batch size if GPU allows
    per_device_eval_batch_size=16,
    num_train_epochs=3,           # Reduce epochs to prevent overfitting
    weight_decay=0.02,            # Slightly increased weight decay
    logging_dir="./logs",
    logging_steps=50,             # More frequent logging
    save_strategy="epoch",       # Save model at each epoch
    save_total_limit=2,           # Limit saved models to save space
    load_best_model_at_end=True,  # Automatically load the best model
    metric_for_best_model="loss", # Validation loss as the key metric
    report_to="wandb",           # Log to WandB
)

# Initialize WandB
wandb.init(project="nlp_orientation_task", mode="online")

# Optimizer and Learning Rate Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
steps_per_epoch = len(train_dataset) // training_args.per_device_train_batch_size
num_training_steps = steps_per_epoch * training_args.num_train_epochs

lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.204300,0.187053,0.951848,0.905218,0.927948,0.929712
2,0.110700,0.133541,0.973568,0.941427,0.957228,0.957934
3,0.055800,0.138627,0.974973,0.954207,0.964478,0.964856


TrainOutput(global_step=1587, training_loss=0.17361239043886584, metrics={'train_runtime': 971.6262, 'train_samples_per_second': 52.187, 'train_steps_per_second': 1.633, 'total_flos': 1.334130917308416e+16, 'train_loss': 0.17361239043886584, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.1335408091545105, 'eval_precision': 0.973568281938326, 'eval_recall': 0.9414270500532481, 'eval_f1': 0.9572279371954521, 'eval_accuracy': 0.9579339723109691, 'eval_runtime': 11.5612, 'eval_samples_per_second': 162.44, 'eval_steps_per_second': 10.207, 'epoch': 3.0}
